In [1]:
import os
from pathlib import Path
from pprint import pprint
import re
import shutil
from uuid import uuid4

import numpy as np
import pandas as pd
from pandas import DataFrame

%cd /root/nerve/python
import nerve.tools as tools
from nerve.parser import AssetNameParser
from nerve.database import Database
from nerve.database_test import DatabaseTests
from nerve.specification_base import *
from nerve.validators import *

/root/nerve/python


In [2]:
root = '/tmp/projects'
if Path(root).exists():
    shutil.rmtree(root)
os.makedirs(root)
fullpaths = DatabaseTests().create_files(root)

In [3]:
!tree /tmp/projects

/tmp/projects
├── proj001
│   ├── spec001
│   │   └── pizza
│   │       ├── p-proj001_s-spec001_d-pizza_v001
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0002.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v001_c000-001_f0003.png
│   │       ├── p-proj001_s-spec001_d-pizza_v002
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0002.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0003.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0004.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0002.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0003.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v002_c000-001_f0004.png
│   │       └── p-proj001_s-spec001_d-pizza_v003
│

In [4]:
class Spec001(SequenceSpecificationBase):
    name = 'spec001'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'coordinate', 'frame', 'extension']
    coordinate = IntType(required=True, validators=[is_coordinate])

class Spec002(SequenceSpecificationBase):
    name = 'spec002'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'frame', 'extension']
    width = IntType(required=True, validators=[lambda x: is_equal(x, 1024)])
    height = IntType(required=True, validators=[lambda x: is_equal(x, 1024)])
    extension = StringType(required=True, validators=[is_extension, lambda x: is_equal(x, 'exr')])
    
class Vdb001(FileSpecificationBase):
    name = 'vdb001'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'extension']
    extension = StringType(required=True, validators=[is_extension, lambda x: is_equal(x, 'vdb')])

db = Database(root, [Spec001, Spec002, Vdb001])
db.update()
data = db.data
data.head()
data

,project,specification,descriptor,version,coordinate,frame,extension,filename,fullpath,errors,asset_name,asset_path,asset_type
0,proj001,spec001,pizza,1.0,"[0, 1]",1.0,png,p-proj001_s-spec001_d-pizza_v001_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,{},p-proj001_s-spec001_d-pizza_v001,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
1,proj001,spec001,pizza,1.0,"[0, 1]",2.0,png,p-proj001_s-spec001_d-pizza_v001_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,{},p-proj001_s-spec001_d-pizza_v001,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
2,proj001,spec001,pizza,1.0,"[0, 1]",3.0,png,p-proj001_s-spec001_d-pizza_v001_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,{},p-proj001_s-spec001_d-pizza_v001,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
3,proj001,spec001,pizza,2.0,"[0, 0]",1.0,png,p-proj001_s-spec001_d-pizza_v002_c000-000_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,{},p-proj001_s-spec001_d-pizza_v002,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
4,proj001,spec001,pizza,2.0,"[0, 0]",2.0,png,p-proj001_s-spec001_d-pizza_v002_c000-000_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,{},p-proj001_s-spec001_d-pizza_v002,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
5,proj001,spec001,pizza,2.0,"[0, 0]",3.0,png,p-proj001_s-spec001_d-pizza_v002_c000-000_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,{},p-proj001_s-spec001_d-pizza_v002,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
6,proj001,spec001,pizza,2.0,"[0, 0]",4.0,png,p-proj001_s-spec001_d-pizza_v002_c000-000_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,{},p-proj001_s-spec001_d-pizza_v002,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
7,proj001,spec001,pizza,2.0,"[0, 1]",1.0,png,p-proj001_s-spec001_d-pizza_v002_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,{},p-proj001_s-spec001_d-pizza_v002,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
8,proj001,spec001,pizza,2.0,"[0, 1]",2.0,png,p-proj001_s-spec001_d-pizza_v002_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,{},p-proj001_s-spec001_d-pizza_v002,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
9,proj001,spec001,pizza,2.0,"[0, 1]",3.0,png,p-proj001_s-spec001_d-pizza_v002_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,{},p-proj001_s-spec001_d-pizza_v002,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence


In [8]:
data.apply(lambda x: x.to_dict(), axis=1)
data.groupby('asset_path').count()

,project,specification,descriptor,version,coordinate,frame,extension,filename,fullpath,errors,asset_name,asset_id,asset_type
asset_path,,,,,,,,,,,,,
/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v001,3,3,3,3,3,3,3,3,3,3,3,3,3
/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v002,8,8,8,8,8,8,8,8,8,8,8,8,8
/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v003,2,4,2,2,2,2,4,4,4,4,2,2,4
/tmp/projects/proj001/spec002/taco/p-proj001_s-spec002_d-taco_v001,3,3,3,3,0,3,3,3,3,3,3,3,3
/tmp/projects/proj001/spec002/taco/p-proj001_s-spec002_d-taco_v002,1,4,1,1,0,1,4,4,4,4,1,1,4
/tmp/projects/proj002/vdb001/p-proj002_s-vdb001_d-bagel_v001.vdb,1,1,1,1,0,0,1,1,1,1,1,1,1
/tmp/projects/proj002/vdb001/p-proj002_s-vdb001_d-bagel_v002.vdb,1,1,1,1,0,0,1,1,1,1,1,1,1
/tmp/projects/proj002/vdb001/p-proj002_s-vdb001_d-bagel_v003.vdb,1,1,1,1,0,0,1,1,1,1,1,1,1
